In [33]:
def set_hadoop_config_with_credentials(name):
    prefix = 'fs.swift2d.service.' + name
    hconf = sc._jsc.hadoopConfiguration()
    hconf.set(prefix + '.auth.url', 'https://identity.open.softlayer.com'+'/v3/auth/tokens')
    hconf.set(prefix + '.auth.endpoint.prefix', 'endpoints')
    hconf.set(prefix + '.tenant', 'a9fb4d478e3d40a8bbd54c5a2ecf25a3')
    hconf.set(prefix + '.username', '6a4cc8251c1940179a6cccc9098a15e0')
    hconf.set(prefix + '.password', 'kDTcKA2H(3eo5.G0')
    hconf.setInt(prefix + '.http.port', 8080)
    hconf.set(prefix + '.region', 'dallas')
    hconf.setBoolean(prefix + '.public', True)

name = 'keystone'
set_hadoop_config_with_credentials(name)

from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import DoubleType

case1 = spark.read.csv("swift2d://MGH." + name + "/csv/csv_case1.csv",\
                       inferSchema=True,nanValue=0,positiveInf=0,negativeInf=0)

In [34]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import lit

def replaceStrings(c, v):
    is_string = c.isin([
        lit("Inf").cast("string"),
        lit("NaN").cast("string")
    ])
    return when(is_string, v).otherwise(c)

for col in case1.schema:
    if col.dataType == StringType():
        #if the column is of string type
        #if the string cannot be converted to a number, make it 0, if it can, make it that number
        case1 = case1.withColumn(col.name + "temp", replaceStrings(case1[col.name], lit(0)).cast(DoubleType()))
        case1 = case1.drop(col.name).withColumnRenamed(col.name + "temp",col.name)

In [35]:
import numpy as np
case1 = case1.replace(float(np.nan), 0)

In [36]:
from pyspark.sql.functions import monotonically_increasing_id 

case1 = case1.withColumn("timeId", monotonically_increasing_id())
case1.where(case1["_c20"] == 0.0).take(1)

[Row(_c0=2.9649e-81, _c1=2.8679e-81, _c2=3.4059e-81, _c3=8.5881e-81, _c4=1.0068e-83, _c5=9.9952e-84, _c6=1.0632e-83, _c7=2.5581e-83, _c8=8.5101e-85, _c9=8.5399e-85, _c10=9.0173e-85, _c11=1.965e-84, _c12=2.5165e-85, _c13=2.5379e-85, _c14=2.704e-85, _c15=5.6671e-85, _c18=42.49, _c21=0.0, _c22=0.0, _c24=0.0, _c25=0.0, _c27=0.0, _c28=0.0, _c29=0.0, _c30=0.0, _c31=0.0, _c32=7.223e-74, _c33=7.7969e-74, _c34=5.8326e-74, _c35=7.132e-74, _c36=1.3698e-76, _c37=1.451e-76, _c38=1.702e-76, _c39=2.3106e-76, _c40=4.566e-78, _c41=4.7775e-78, _c42=6.8622e-78, _c43=1.3503e-77, _c44=2.5752e-79, _c45=2.6599e-79, _c46=4.5523e-79, _c47=1.0969e-78, _c48=6.9388e-160, _c49=1.1405e-159, _c50=1.0817e-159, _c51=5.3725e-159, _c52=3.7101e-165, _c53=6.0981e-165, _c54=5.7841e-165, _c55=2.8726e-164, _c56=1.2141e-167, _c57=1.9956e-167, _c58=1.8929e-167, _c59=9.4003e-167, _c60=3.806e-169, _c61=6.2559e-169, _c62=5.9338e-169, _c63=2.9469e-168, _c64=3.4455e-159, _c65=5.6633e-159, _c66=5.3714e-159, _c67=2.6678e-158, _c68=0.

In [37]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import UserDefinedFunction
from pyspark.ml.feature import VectorAssembler 

feature_cols = list(filter(lambda x:x not in ['id'],case1.columns))
assembler = VectorAssembler(inputCols=feature_cols,outputCol='features')
df = assembler.transform(case1)

pca = PCA(k=100, inputCol="features", outputCol="pcaFeatures")
pcaModel = pca.fit(df)


pca_result = pcaModel.transform(df)
pca_result.select("pcaFeatures").take(1)

[Row(pcaFeatures=DenseVector([-609244.2378, -84386.2173, -33814.2339, -39512.1189, 30360.2774, -30540.0115, 10524.6997, -24277.5839, 4992.7956, 13210.207, -4352.5545, 6500.288, 613.2874, -3242.7923, -4747.9424, -305.3784, -570.896, 1102.4275, 1288.4616, -1413.4762, 658.3513, -338.1213, -158.4801, -23.682, 29.623, -26.29, 117.8277, 21.2811, -49.6724, 96.8796, -7.8959, 33.8028, 81.2327, 15.1889, 46.726, -8.013, 3.2356, 36.9925, -18.5134, 15.3142, -55.2419, 23.5964, 13.3107, -33.2705, 28.5732, 2.7267, -11.5178, 9.2158, -41.9234, -6.5597, 11.4569, 6.7738, 30.4465, -46.1943, -46.2712, -6.4496, -82.9544, -55.9177, -11.9147, 6.1268, -3.4021, 6.9592, 15.2449, -18.4333, -13.9071, -3.3836, -92.229, -8.6641, -122.5631, 29.8729, -27.8781, -13.8965, 14.0109, -1.8602, 14.2568, 14.8893, -23.5304, 3.4506, -59.7862, -36.7106, -6.5657, 71.4061, -2.829, -30.0952, 47.5812, -8.9283, -1.437, 5.7676, -6.1115, -12.2393, -2.1296, -8.5044, -25.7721, -25.7626, -9.8018, -5.3744, 10.9543, 6.9348, -3.943, 2.4883]))

In [39]:
pca_result.take(1)

[Row(_c0=2.2543, _c1=2.2802, _c2=3.2058, _c3=2.7831, _c4=2.2414, _c5=2.3961, _c6=3.1904, _c7=2.8007, _c8=2.2199, _c9=2.3412, _c10=3.0924, _c11=2.7539, _c12=1.973, _c13=2.2811, _c14=3.0617, _c15=2.9103, _c18=23.097, _c21=3.1503, _c22=3.0308, _c24=3.4753, _c25=2.9078, _c27=3.1089, _c28=3.6284, _c29=4.1932, _c30=3.0422, _c31=3.6411, _c32=254330.0, _c33=226880.0, _c34=341440.0, _c35=284210.0, _c36=109680.0, _c37=113230.0, _c38=125590.0, _c39=124770.0, _c40=53409.0, _c41=56575.0, _c42=60883.0, _c43=60030.0, _c44=12915.0, _c45=13836.0, _c46=13558.0, _c47=15694.0, _c48=62.59, _c49=56.045, _c50=118.16, _c51=97.662, _c52=49.497, _c53=53.792, _c54=87.745, _c55=83.527, _c56=47.886, _c57=54.141, _c58=82.052, _c59=77.989, _c60=42.201, _c61=45.07, _c62=73.033, _c63=81.49, _c64=230.36, _c65=149.73, _c66=464.07, _c67=249.64, _c68=74.399, _c69=79.464, _c70=128.04, _c71=125.21, _c72=72.525, _c73=80.443, _c74=116.56, _c75=114.27, _c76=60.959, _c77=59.875, _c78=98.354, _c79=111.16, _c80=0.8756, _c81=0.861

In [41]:
pca_result.write.parquet("swift2d://MGH." + name + "/tempParq/case1.parquet")

In [2]:
# See ls listing

!ls -l


total 2881368
drwxr-x--- 2 seff-34c2f0d3dcc620-a916a00b641d users      4096 May 26 13:45 Case1
-rw-r----- 1 seff-34c2f0d3dcc620-a916a00b641d users 983503825 May 26 11:18 Case1_seg1.mat
-rw-r----- 1 seff-34c2f0d3dcc620-a916a00b641d users       800 May 26 17:06 MATLAB 7.3 MAT-file, Platform: PCWIN64, Created on: Wed May 24 13:46:39 2017 HDF5 schema 1.00 .                     
-rw-r----- 1 seff-34c2f0d3dcc620-a916a00b641d users       800 May 26 12:31 MATLAB 7.3 MAT-file, Platform: PCWIN64, Created on: Wed May 24 13:54:28 2017 HDF5 schema 1.00 .                     
drwx------ 3 seff-34c2f0d3dcc620-a916a00b641d users      4096 Jun  5 09:02 MGH.keystone
drwx------ 2 seff-34c2f0d3dcc620-a916a00b641d users      4096 Jun  4 22:55 MGH-TEST-01
drwxr-x--- 2 seff-34c2f0d3dcc620-a916a00b641d users      4096 May 31 12:05 spark-warehouse
cost.txt  result00001.csv


In [3]:
!ls -l MGH-TEST-01

total 6312
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users       0 Jun  5 09:11 cost.txt
-rw-r--r-- 1 seff-34c2f0d3dcc620-a916a00b641d users 2151689 Jun  5 09:47 result00001.csv
